# Where is CDBG money being spent in Los Angeles?

In [3]:
import json

import altair
import intake
import ipyleaflet
import ipywidgets
import pandas
import geopandas
from IPython.display import Markdown

cat = intake.open_catalog("../catalogs/*.yml")

In [17]:
# Download geospatial data

city = cat.la_geohub.city_boundary.read()
county = cat.la_geohub.county_boundary.read()
cpa = cat.la_geohub.community_plan_area.read().rename(columns={"NAME_ALF": "cpa"})
apc = cat.la_geohub.area_planning_commision.read().rename(columns={"Name": "apc"})
council = cat.la_geohub.council_districts.read().rename(columns={"District_Name": "council_district"})
neighborhood = cat.la_geohub.neighborhood_councils.read().rename(columns={"Name": "neighborhood_council"})

In [5]:
# Download geocoded PEP data

pep = cat.pep_geocode.read()

In [11]:
# Download Low/medium income data from HUD at the block group level

lmi_hud = cat.lmi_hud.read()
lmi_hud = lmi_hud[lmi_hud.intersects(city.iloc[0].geometry)]

In [12]:
# Filter out PEPs that are not likely to be locatable on a map, whether
# they are city-wide, confidential, or located in many places.
unlocatable = [
    "confidental",
    "confidential",
    "various",
    "citywide",
    "city-wide",
    "n/a",
    "see attached"
]
unlocatable_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains('|'.join(unlocatable)) |
    (pep.citywide_cncl_dist_yn == 1)
]

# The rest of the PEPs *may* be locatable.
located_pep = pep[
    ~pep.pep_proj_loc_addr.str.lower().str.contains('|'.join(unlocatable)) &
    ~(pep.citywide_cncl_dist_yn == 1)
]

In [13]:
# If the geocoded location of a PEP falls outside of the city geometry,
# remove it.
pep_in_city = located_pep.within(city.iloc[0].geometry)
unlocated_pep = located_pep[~pep_in_city]

# Consider the rest of the PEPs to be properly located
located_pep = located_pep[pep_in_city]

In [19]:
# For reporting purposes, we also want to know which PEPs are for
# confidential or city-wide programs, so aggregate those on a yearly basis.
confidential_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains("confidental|confidential") &
    (pep.citywide_cncl_dist_yn == 1)
].groupby("year").agg({"cdbg_fnd_amt": lambda x: sum(x)/1.e6}).reset_index().assign(
    name="Confidential"
).rename(columns={"cdbg_fnd_amt": "amount"})
citywide_pep = pep[
    pep.pep_proj_loc_addr.str.lower().str.contains("citywide|city-wide") &
    (pep.citywide_cncl_dist_yn == 1)
].groupby("year").agg({"cdbg_fnd_amt": lambda x: sum(x)/1.e6}).reset_index().assign(
    name="Citywide"
).rename(columns={"cdbg_fnd_amt": "amount"})

In [20]:
# Do spatial joins with all of the geometries that we might want to aggregate
# up to, including city council districts, community plan areas, area planning
# commision, and neighborhood councils.
pep_joined = geopandas.sjoin(
    located_pep,
    cpa[["cpa", "geometry"]],
    how="left",
    op="within",
).drop(columns=["index_right"])
pep_joined = geopandas.sjoin(
    pep_joined,
    apc[["apc", "geometry"]],
    how="left",
    op="within",
).drop(columns=["index_right"])
pep_joined = geopandas.sjoin(
    pep_joined,
    neighborhood[["neighborhood_council", "geometry"]],
    how="left",
    op="within",
).drop(columns=["index_right"])
pep_joined = geopandas.sjoin(
    pep_joined,
    council[["council_district", "geometry"]],
    how="left",
    op="within",
).drop(columns=["index_right"])

In [21]:
# Create the LMI choropleth layer
def get_lmi_choro_layer():
    choro_data = lmi_hud.Lowmod_pct.to_dict()
    choro_data = {str(x): y for x,y in choro_data.items()}
    geo_data = json.loads(lmi_hud.to_json())

    choro_layer = ipyleaflet.Choropleth(
        style={'fillOpacity': 0.6, "weight": 0},
        geo_data=geo_data,
        choro_data=choro_data,
    )
    return choro_layer
lmi_choro_layer = get_lmi_choro_layer()

In [22]:
# Create the base map
m = ipyleaflet.Map(basemap=ipyleaflet.basemaps.CartoDB.Positron)

# Create the choropleth layer for a given geographic unit
def get_choro_layer(year, geog):
    # map dropdown values to geodataframes
    geo = {
        "cpa": cpa,
        "neighborhood_council": neighborhood,
        "apc": apc,
        "council_district": council,
    }
    # filter to the requested years
    projects = pep_joined[
        (pep_joined.year >= year[0]) &
        (pep_joined.year <= year[1])
    ]
    # Aggregate funding according to the geometry
    agg = projects.groupby(geog).agg({
        "cdbg_fnd_amt": lambda x: sum(x)/1.e6
    }).rename(columns={"cdbg_fnd_amt": "amount"})
    df = geo[geog].set_index(geog).assign(
        amount=agg.amount
    ).fillna(0)[
        ["geometry", "amount"]
    ]

    # Create the choropleth layer
    choro_data = df.amount.to_dict()
    geo_data = json.loads(df.to_json())
    choro_layer = ipyleaflet.Choropleth(
        style={'fillOpacity': 0.6},
        geo_data=geo_data,
        choro_data=choro_data,
        vmin=0.0,
        vmax=agg.amount.max()
    )
    return choro_layer

# Create an initial choropleth layer.
init_year = (36, 44)
init_geog = "apc"
choro_layer = get_choro_layer(init_year, init_geog)
m.add_layer(choro_layer)

# Create the individual project layer
circle_layer = ipyleaflet.LayerGroup(markers=[])
def update_circles(year):
    circles = []
    projects = pep_joined[
        (pep_joined.year >= year[0]) &
        (pep_joined.year <= year[1])
    ]
    for idx, row in projects.iterrows():
        circles.append(ipyleaflet.Circle(
            location=(row.geometry.y, row.geometry.x),
            radius=int(row.cdbg_fnd_amt/5.e3),
            weight=1,
            popup=ipywidgets.HTML(value=f"""
            <b>Project: </b>{row.pep_proj_nm} <br>
            <b>Agency: </b>{row.pep_agcy_nm} <br>
            <b>Address: </b>{row.address} <br>
            <b>Fund amount </b>${row.cdbg_fnd_amt:,.0f} <br>
            """),
        ))
    circle_layer.layers=circles
m.add_layer(circle_layer)

# Create the hover label
region_type = {
    "cpa" : "Community Plan Area",
    "apc": "Area Planning Commision",
    "neighborhood_council": "Neighborhood Council",
    "council_district": "Council District",
}
label = ipywidgets.HTML(value=f"<b>{region_type[init_geog]}: </b><i>Hover to select</i>")

# Create the Low/Medium income checkbox
lmi_box = ipywidgets.Checkbox(
    description="Show Low/Medium Income Percent",
    value=True,
    style = {'description_width': 'initial'},
)

# Callback to update the map data when the slider is changed
def update_map(year, geog):
    if lmi_box.value:
        layer = lmi_choro_layer
        def on_hover(**kwargs):
            properties = kwargs.get("feature", {}).get("properties")
            id = kwargs.get("feature", {}).get("id")
            if not properties:
                return
            label.value=f"""
            <b>Percent Low/Medium income: </b> {properties["Lowmod_pct"]} <br>
            """
        label.value=f"<b>Percent Low/Medium income: </b> <i>Hover to select</i>"
        layer.on_hover(on_hover)
    else:
        def on_hover(**kwargs):
            properties = kwargs.get("feature", {}).get("properties")
            id = kwargs.get("feature", {}).get("id")
            if not properties:
                return
            label.value=f"""
            <b>{region_type[geog]}: </b> {id} <br>
            <b>Total funding: </b> ${properties['amount']:,.1f}M
            """
        label.value=f"<b>{region_type[geog]}: </b><i>Hover to select</i>"
        layer = get_choro_layer(year, geog)
        layer.on_hover(on_hover)

    for l in m.layers:
        if isinstance(l, ipyleaflet.Choropleth):
            m.substitute_layer(l, layer)
            break
    else:
        m.add_layer(layer)
    update_circles(year)

# Create the geography slider
geography = ipywidgets.Dropdown(
    description="Geography",
    options=[
        ("Community Plan Area", "cpa"),
        ("Area Planning Commission", "apc"),
        ("Council District", "council_district"),
        ("Neighborhood Council", "neighborhood_council"),
    ],
    value=init_geog,
)
ipywidgets

# Create the year slider
slider = ipywidgets.SelectionRangeSlider(
    description="Program Years",
    options=[
        (f"{1974+x}/{1975+x}", x)
        for x in range(pep_joined.year.min(), pep_joined.year.max()+1)
    ],
    style = {"description_width": 'initial'},
    value=(44,44),
)
slider.layout.width="512px"

# Add controls
m.add_control(ipyleaflet.WidgetControl(widget=slider, position="topright"))
m.add_control(ipyleaflet.WidgetControl(widget=geography, position="topright"))
m.add_control(ipyleaflet.WidgetControl(widget=lmi_box, position="topright"))
m.add_control(ipyleaflet.WidgetControl(widget=label, position="topright"))


# Finialize the map.
update_map(slider.value, geography.value)
m.center = [34.07996230865876, -118.31123326410754]
m.zoom = 10

In [23]:
hbox = ipywidgets.HBox()
o1 = ipywidgets.Output()
o2 = ipywidgets.Output()

def update_charts(*args):
    agg = pep_joined.groupby([geography.value, "year"]).agg({
            "cdbg_fnd_amt": lambda x: sum(x)/1.e6
        }).reset_index(level=1).rename(columns={"cdbg_fnd_amt": "amount"})
    to_chart = pandas.concat([
        agg.reset_index(),
        confidential_pep.rename(columns={"name": geography.value}),
        citywide_pep.rename(columns={"name": geography.value}),
    ])
    with o1:
        o1.clear_output(wait=True)
        areas = to_chart.groupby(geography.value).amount.sum().sort_values().tail(10).index
        display(
            altair.Chart(
                to_chart[to_chart[geography.value].isin(areas)]
            ).mark_line(strokeWidth=4).encode(
                x=altair.X("year", title="Grant year"),
                y=altair.Y("amount", title="Funding amount (million $)"),
                color=altair.Color(geography.value, legend=altair.Legend(
                    direction="vertical",
                    orient="top",
                    title=region_type[geography.value],
                )),
                tooltip=[geography.value, "amount", "year"],
            )
        )
    with o2:
        o2.clear_output(wait=True)
        display(
            to_chart
            .rename(columns={geography.value: region_type[geography.value]})
            .groupby(region_type[geography.value])
            .agg({"amount": "sum"})
            .sort_values("amount", ascending=False)
            .rename(columns={"amount": "Total funding amount"})
            .style.format(lambda x: f"${x:,.1f} million")
        )

    hbox.children = [o1, o2]
    
update_charts()

In [24]:
# Callback for interacting with controls
def on_selection(*args):
    update_map(slider.value, geography.value)
    update_charts()

slider.observe(on_selection, names=["value"])
geography.observe(on_selection, names=["value"])
lmi_box.observe(on_selection, names=["value"])


# Show the map
display(m)
display(hbox)

Map(center=[34.07996230865876, -118.31123326410754], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [25]:
display(Markdown(
    f"Number of confidential, city-wide, or otherwise unlocatable PEPs: {len(unlocatable_pep)}"
))
display(Markdown(
    f"Number of unlocated PEPs, or ones outside of the City of LA: {len(unlocated_pep)}"
))
display(Markdown(
    f"Number of located PEPs: {len(located_pep)}"
))

Number of confidential, city-wide, or otherwise unlocatable PEPs: 934

Number of unlocated PEPs, or ones outside of the City of LA: 126

Number of located PEPs: 1803